In [ ]:
# 시스템 모듈
import os
from dotenv import load_dotenv

# 랭체인 모듈
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory 
from langchain_core.prompts import MessagesPlaceholder

# 환경변수 로드
load_dotenv()

# 벡터 DB 설정
DB_INDEX = "MANUAL_DB"
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local(DB_INDEX, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# LLM 모델 설정
llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0,
    streaming=True,
)

# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system",
    """
    Answer the question using ONLY the following context.
    If you don't know the answer just say you don't know. DON'T make anything up.

    Context: {context}
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

# 검색 증강 문서 처리
def formatting_docs(docs):
    unique_sources = set()
    context = []
    for doc in docs:
        source = doc.metadata.get("source", "")
        if source not in unique_sources and os.path.exists(source):
            unique_sources.add(source)
            with open(source, 'r', encoding='utf-8') as file:
                content = file.read()
                context.append(content)
    
    return "\n\n".join(context)

# 대화 메모리 설정
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# 체인 설정
chain = {
    "context": retriever | RunnableLambda(formatting_docs),
    "chat_history": RunnableLambda(lambda _: memory.load_memory_variables({}).get('chat_history', [])),
    "question": RunnablePassthrough()
} | prompt | llm 

# 대화 메모리를 사용한 입력 및 출력 관리
def run_chain_with_memory(query):
    # 체인을 사용하여 응답 생성
    response = chain.invoke(query)
    # 대화 메모리에 응답 추가
    memory.save_context(
        inputs={"human": query},
        outputs={"ai": response.content}
    )
    return response


In [64]:
query = "김장호가 누구인가요?"
response = run_chain_with_memory(query)

print(response)

김장호는 26세의 서비스 기획자입니다.


In [65]:
query = "내가 방금전에 어떤 내용을 물어보았나요?"
response = run_chain_with_memory(query)

print(response)

당신이 방금 전에 김장호가 누구인지 물어보았습니다.


In [66]:
print(memory.load_memory_variables({}).get('chat_history'))



[HumanMessage(content='내가 방금전에 어떤 내용을 물어보았나요?'), AIMessage(content='당신이 방금 전에 어떤 내용을 물어보았는지에 대한 정보는 제공된 컨텍스트에 포함되어 있지 않습니다.'), HumanMessage(content='김장호가 누구인가요?'), AIMessage(content='김장호는 26세의 서비스 기획자입니다.'), HumanMessage(content='내가 방금전에 어떤 내용을 물어보았나요?'), AIMessage(content='당신이 방금 전에 김장호가 누구인지 물어보았습니다.')]
